In [1]:
# -*- coding: UTF-8 -*-

# author: s0nnet
# time: 2016-11-08
# desc: MySQL基础操作类封装


import MySQLdb
from bs_log import *

class SqlCmdEnum:
    COMMIT_CMD = 0
    ROLLBACK_CMD = 1
    
class SqlExecEnum:
    QUERY       =   0
    UPDATE      =   1
    INSERT      =   2
    DELETE      =   3
    

def _new_conn(host, port, user, passwd, db, charset):
    try:
        conn = MySQLdb.connect(host = host,
                               port = port,
                               user = user,
                               passwd = passwd,
                               db = db,
                               charset = charset)
        conn.autocommit(True)
        Log.debug("conn(host: %s, port: %d, user: %s, passwd: %s, db: %s)" %(host, port, user, passwd, db))
        return conn
    except Exception, e:
        Log.err("ERR(host: %s, port: %d, user: %s, passwd: %s, db: %s, %s)" %(host, port, user, passwd, db, str(e)))
        return None

class DataBase: 
    
    def __init__(self,               
                 host = BaseConf.SQL_HOST, 
                 port = BaseConf.SQL_PORT,
                 user = BaseConf.SQL_USER,
                 passwd = BaseConf.SQL_PASSWD,
                 db = BaseConf.SQL_DB,
                 charset = None):
        self.host = host
        self.port = port
        self.user = user
        self.passwd = passwd
        self.db = db
        self.charset = charset
        
        self.conn = None
        
    def _check(self):
        try:
            self.conn.ping()
        except Exception, e:
            self.conn = _new_conn(self.host, self.port, self.user, self.passwd, self.db, self.charset)
        
    def _get_conn(self):
        if self.conn == None:
            self.conn = _new_conn(self.host, self.port, self.user, self.passwd, self.db, self.charset)
        else: 
            self._check()
    
    def _close_conn(self):
        if self.conn != None:
            self.conn.commit()
            self.conn.close()
            self.conn = None
            

    def _get_record_list(self, cursor):
        
        def _get_fieldname_list(cursor):
            fieldList = []
            for info in cursor.description:
                fieldList.append(info[0])
            return fieldList
        
        fieldnameList = _get_fieldname_list(cursor)
        
        retNodeList = []
        recordList = cursor.fetchall()
        for record in recordList:
            node = {}
            for i in range(len(fieldnameList)):
                node[fieldnameList[i]] = record[i]
            retNodeList.append(node)
        
        return retNodeList
            
        
    def _exec_cmdstr(self, cmdstr, param, rollback, execType):
        
        def _encode_param(param):
            dParam = []
            for par in param:
                if type(par) == unicode:
                    par = par.encode('utf-8')
                dParam.append(par)
            return dParam
        
        if param != None:  param = _encode_param(param)
        Log.debug("%s\t%s" %(cmdstr, str(param)))
        
        self._get_conn()
        conn = self.conn
        if conn == None:
            Log.err("mysql not connect!")
            return False, "mysql not connect!"
        
        cursor = conn.cursor()
        try:
            val = cursor.execute(cmdstr, param)
            if execType == SqlExecEnum.QUERY:    val = self._get_record_list(cursor)
            elif execType == SqlExecEnum.INSERT: val = int(conn.insert_id())
            else: pass
        except Exception, e:
            Log.warning("sql: %s,%s ERR(%s)" %(cmdstr, str(param), str(e)))
            if rollback:
                self.cmd_submit(SqlCmdEnum.ROLLBACK_CMD)
            else:
                self._close_conn()        
            return False, "%s:%s %s" %(cmdstr, str(param), str(e))

        if not rollback:
            self._close_conn()
        
        return True, val
    
    def query_data(self, cmdstr, param, rollback = False):
        return self._exec_cmdstr(cmdstr, param, rollback, SqlExecEnum.QUERY)
    
    def update_data(self, cmdstr, param, rollback = False):
        return self._exec_cmdstr(cmdstr, param, rollback, SqlExecEnum.UPDATE)
        
    def insert_data(self, cmdstr, param, rollback = False):
        return self._exec_cmdstr(cmdstr, param, rollback, SqlExecEnum.INSERT)
    
    def delete_data(self, cmdstr, param, rollback = False):
        return self._exec_cmdstr(cmdstr, param, rollback, SqlExecEnum.DELETE)
    
    def cmd_submit(self, flag):
        conn = self.conn
        if not conn: self._close_conn(); return
        
        try:
            if flag == SqlCmdEnum.COMMIT_CMD:
                conn.commit()
            elif flag == SqlCmdEnum.ROLLBACK_CMD:
                conn.rollback()
        finally:
            self._close_conn()
            

if __name__ == "__main__":
    
    dataBase = DataBase()
    
    cmdstr = "select * from tb_user limit 10"
    print dataBase.query_data(cmdstr, None)


(True, [])


In [1]:
%%file Arthur_utils.py
# token类
# @since 2018-12-20
class Connector:
    # 加密 token
    @classmethod
    def encrypt_token(cls, uid, access_token, session_token):
        if access_token == "" or session_token == "":
            return {}

        uid = int(uid)
        if uid < 17:  # 防止出现 id 过小的情况
            uid = uid * 19

        if len(access_token) < 32:  # 防止出现长度不足的情况
            access_token = Itaken.enmd5(access_token)
        if len(session_token) < 32:
            session_token = Itaken.enmd5(session_token)

        session_token_list = list(session_token)  # token string转list
        str_key = access_token[8:20]  # 字符替换key

        pos = []
        mod_key = (11, 3, 7, 13, 5, 17)  # 求模key
        for index, i in enumerate(mod_key):
            chart = access_token[index: index + 1]  # 随机数
            point = (uid + int("0x" + chart, 16)) % i  # 获取的位置(16进制转10进制)

            pos = Itaken.spread_list(point, pos, 3, 1)  # 计算前半段
            pos = Itaken.spread_list(point + 16, pos, 3, 1)  # 计算后半段

        for index, i in enumerate(pos):
            session_token_list[i] = str_key[index]  # 密钥替换

        token = "".join(session_token_list)  # 加密后token

        return {
            "position": pos,
            "key": str_key,
            "token": token
        }

    # 验证 token
    #TODO
    @classmethod
    def verify_token(cls, uid, token):
        uid = int(uid)
        if uid < 1 or token == "" or token is None:
            return {"code": 0, "message": "验证内容不合法"}
        #TODO
        token_info = cls.get_user_token_info(uid=uid)  # 获取数据库token信息
        if not token_info:
            return {"code": 0, "message": "没有该用户"}

        encrypt_info = cls.encrypt_token(uid=uid, access_token=token_info["access_token"], session_token=token_info["session_token"])
        if token == encrypt_info["token"]:
            return {"code": 1, "message": "验证成功", "uid": uid}

        token_list = list(token)  # token 转list
        token_key = ""
        for i in encrypt_info["position"]:
            token_key += token_list[i]  # 组装密钥

        if token_key == encrypt_info["key"]:
            return {"code": -1, "message": "session token失效"}

        return {"code": -2, "message": "access token失效"}

Writing Arthur_utils.py


In [1]:
def _get_log_tm():
    "[01/Jan/2013 04:51:46]"
    return (datetime.datetime.now()).strftime("[%d/%m/%Y %H:%M:%S]")
import datetime
_get_log_tm()

'[25/01/2019 08:49:54]'